# Text Selection

In this section we'll try text selection...

In [3]:
%load_ext autoreload
%autoreload 2
import sys
import os
from tqdm.auto import tqdm
sys.path.insert(0,'../')

from src.legifrance import LegifranceClient,LegifranceDatabase
database = LegifranceDatabase(path='../data/sqlite/legifrance.db')
# database.create_database(overwrite=True)

laws = database.get_lawtexts('code du travail')
laws

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[LawText(legi_id='LEGITEXT000006072050', title=Code du travail),
 LawText(legi_id='LEGITEXT000006072051', title=Code du travail maritime),
 LawText(legi_id='LEGITEXT000006072052', title=Code du travail applicable à Mayotte)]

In [4]:
for law in laws:
    articles = law.get_all_articles()
    for article in tqdm(articles):
        article.content = None
        article.title = None
    database.commit()

  0%|          | 0/23363 [00:00<?, ?it/s]

  0%|          | 0/191 [00:00<?, ?it/s]

  0%|          | 0/3502 [00:00<?, ?it/s]

In [22]:
from src.legifrance.orm import Article,Section,LawText
from sqlalchemy import select

select_texts = select(LawText)
texts = database.session.execute(select_texts).all()
texts

[(LawText(legi_id='LEGITEXT000006071785', title=Code minier),),
 (LawText(legi_id='LEGITEXT000006070721', title=Code civil),),
 (LawText(legi_id='LEGITEXT000045476241', title=Code pénitentiaire),),
 (LawText(legi_id='LEGITEXT000006070719', title=Code pénal),),
 (LawText(legi_id='LEGITEXT000006070239', title=Code électoral),),
 (LawText(legi_id='LEGITEXT000006070162', title=Code des communes),),
 (LawText(legi_id='LEGITEXT000006074073', title=Code du tourisme),),
 (LawText(legi_id='LEGITEXT000006071570', title=Code des douanes),),
 (LawText(legi_id='LEGITEXT000006075116', title=Code de l'artisanat),),
 (LawText(legi_id='LEGITEXT000006073984', title=Code des assurances),),
 (LawText(legi_id='LEGITEXT000006074236', title=Code du patrimoine),),
 (LawText(legi_id='LEGITEXT000006071318', title=Code du sport),),
 (LawText(legi_id='LEGITEXT000005634379', title=Code de commerce),),
 (LawText(legi_id='LEGITEXT000023983208', title=Code de l'énergie),),
 (LawText(legi_id='LEGITEXT000006074075', ti

In [3]:
from sqlalchemy import select,and_
for result in texts:
    text = result[0]
    text:LawText
    sections = text.get_all_sections()
    section_ids = [s.id for s in sections]
    articles_query = select(Article).where(and_(
        Article.content.is_(None),
        Article.section_id.in_(section_ids)
        )
    )
                
    articles = database.session.execute(articles_query).all()
    print(articles)

[(Article(legi_id='LEGIARTI000006627338', title=None),), (Article(legi_id='LEGIARTI000022483212', title=None),), (Article(legi_id='LEGIARTI000023518706', title=None),), (Article(legi_id='LEGIARTI000006627347', title=None),), (Article(legi_id='LEGIARTI000006627350', title=None),), (Article(legi_id='LEGIARTI000006627353', title=None),), (Article(legi_id='LEGIARTI000006627358', title=None),), (Article(legi_id='LEGIARTI000006627362', title=None),), (Article(legi_id='LEGIARTI000006627365', title=None),), (Article(legi_id='LEGIARTI000006627368', title=None),), (Article(legi_id='LEGIARTI000006627371', title=None),), (Article(legi_id='LEGIARTI000006627375', title=None),), (Article(legi_id='LEGIARTI000006627377', title=None),), (Article(legi_id='LEGIARTI000006627382', title=None),), (Article(legi_id='LEGIARTI000006627385', title=None),), (Article(legi_id='LEGIARTI000006627391', title=None),), (Article(legi_id='LEGIARTI000006627394', title=None),), (Article(legi_id='LEGIARTI000006627398', title=

KeyboardInterrupt: 

In [65]:
select_texts = select(LawText).where(LawText.legi_id=='LEGITEXT000006070719')
text = database.session.execute(select_texts).first()[0]
text

LawText(legi_id='LEGITEXT000006070719', title=Code pénal)

In [66]:
select_sections = select(Section).where(Section.lawtext==text)
sections = database.session.execute(select_sections).all()

Test suppression en cascade...

In [67]:
select_texts = select(LawText).where(LawText.legi_id=='LEGITEXT000006071785')
text = database.session.execute(select_texts).first()[0]
text

LawText(legi_id='LEGITEXT000006071785', title=Code minier)

In [68]:
text.get_articles()

[Article(legi_id='LEGIARTI000006627108', title=None),
 Article(legi_id='LEGIARTI000006627110', title=None),
 Article(legi_id='LEGIARTI000006627112', title=None),
 Article(legi_id='LEGIARTI000006627113', title=None),
 Article(legi_id='LEGIARTI000022964298', title=None),
 Article(legi_id='LEGIARTI000022494495', title=None),
 Article(legi_id='LEGIARTI000022483143', title=None),
 Article(legi_id='LEGIARTI000021948636', title=None),
 Article(legi_id='LEGIARTI000006627120', title=None),
 Article(legi_id='LEGIARTI000006627121', title=None),
 Article(legi_id='LEGIARTI000006627126', title=None),
 Article(legi_id='LEGIARTI000006627128', title=None),
 Article(legi_id='LEGIARTI000006627130', title=None),
 Article(legi_id='LEGIARTI000006627131', title=None),
 Article(legi_id='LEGIARTI000006627132', title=None),
 Article(legi_id='LEGIARTI000006627133', title=None),
 Article(legi_id='LEGIARTI000006627134', title=None),
 Article(legi_id='LEGIARTI000006627135', title=None),
 Article(legi_id='LEGIARTI00

# Delete items...

In [1]:
import sys
sys.path.insert(0,'../')
from sqlalchemy import delete
from src.legifrance.orm import LawText,CollectiveConvention
from src.legifrance import LegifranceClient,LegifranceDatabase
database = LegifranceDatabase(path='../data/sqlite/legifrance.db')

# delete_query = delete(LawText).where(LawText.legi_id.in_(['LEGITEXT000006072050','LEGITEXT000006072051','LEGITEXT000006072052']))
# database.session.execute(delete_query)
# database.commit()
delete_query = delete(CollectiveConvention)
database.session.execute(delete_query)
database.commit()

In [70]:
select_texts = select(LawText).where(LawText.legi_id=='LEGITEXT000006071785')
database.session.execute(select_texts).first() == None

True

In [71]:
select_articles = select(Article).where(Article.legi_id=='LEGIARTI000006627140')
article = database.session.execute(select_articles).first()[0]
article

TypeError: 'NoneType' object is not subscriptable

In [13]:
client = LegifranceClient()
sections = client.get_lawtext(text)

Code pénal: 0it [00:00, ?it/s]

We check that no double section to create...

In [27]:
articles_1=sections[1].get_articles()
doublons = []

for article in articles_1:
    iso = [a for a in articles_1 if a.legi_id==article.legi_id]
    if len(iso)!=1:
        doublons.append(article)


In [30]:
sections_doublons = [d.section for d in doublons]

sections_doublons[0].articles

[Article(legi_id='LEGIARTI000006419414', title=None),
 Article(legi_id='LEGIARTI000006419415', title=None)]

We check that no section already in database...

In [57]:
section_ids = [s.legi_id for s in sections]
select_existing_sections = select(Section).where(Section.legi_id.in_(section_ids))
existing = database.session.execute(select_existing_sections).all()
print(existing)

[]


We check that no article already in database...

In [58]:
articles = []
for section in sections:
    articles.extend(section.get_articles())

In [59]:
legi_ids = [a.legi_id for a in articles]

existing_article_query = select(Article).where(Article.legi_id.in_(legi_ids))
existing = database.session.execute(existing_article_query).all()
print(existing)

[]


# Test Scrapper

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.insert(0,'../')

from src.legifrance.legifrance_scrapper import LegifranceScrapper
from src.legifrance.orm import Article,Section,LawText
from sqlalchemy import select
from src.legifrance import LegifranceClient,LegifranceDatabase

database = LegifranceDatabase(path='../data/legifrance/database.db')
scrapper = LegifranceScrapper()

query = select(LawText)
text = database.session.execute(query).first()[0]
text:LawText
sections = text.get_all_sections()

In [10]:
articles_list = scrapper.get_section(sections[0].legi_id,text.legi_id)

AttributeError: 'NoneType' object has no attribute 'group'

In [8]:
articles_list

[{'legi_id': 'LEGIARTI000006627108',
  'content': 'Les gîtes de substances minérales ou fossiles renfermés dans le sein de la terre ou existant à la surface sont, relativement à leur régime légal, considérés comme mines ou comme carrières.',
  'title': 'Article 1 (abrogé)'},
 {'legi_id': 'LEGIARTI000006627110',
  'content': "Sont considérés comme mines les gîtes connus pour contenir :\n   - de la houille, du lignite, ou d'autres combustibles fossiles, la tourbe exceptée, des bitumes, des hydrocarbures liquides ou gazeux, du graphite, du diamant ;\n   - des sels de sodium et de potassium à l'état solide ou en dissolution, à l'exception de ceux contenus dans les eaux salées utilisées à des fins thérapeutiques ou de loisirs ;\n   - de l'alun, des sulfates autres que les sulfates alcalino-terreux ;.\n   - de la bauxite, de la fluorine ;\n   - du fer, du cobalt, du nickel, du chrome, du manganèse, du vanadium, du titane, du zirconium, du molybdène, du tungstène, de l'hafnium, du rhénium ;\n